In [22]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import time
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [23]:
import os
import sys

nb_dir_1 = os.path.split(os.getcwd())[0]
current_path = os.getcwd()
parent = os.path.dirname(current_path)
nb_dir_2 = os.path.split(parent)[0]

if nb_dir_1 not in sys.path:
    sys.path.append(nb_dir_1)

if nb_dir_2 not in sys.path:
    sys.path.append(nb_dir_2)

In [24]:
from core.models import LSTM
from core.time_series_clustering import KmeansClustering
from core.data_preparation import DataPreparation

Load and Prepare data to input to LSTM model

In [34]:
filepath = "../Data/MRF_clean.csv"
data = pd.read_csv(filepath, usecols=[0,4], names=['date', 'close'], header=0)
data = data.sort_values('date')
data['date'] = pd.to_datetime(data['date'])
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/MRF_clean'

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['date'], y=data['close'], mode='lines', name='closing price'))

In [27]:
window_size = 20
prep = DataPreparation()
scaler = MinMaxScaler(feature_range=(-1, 1))

In [28]:
scaler, x_train, x_test, y_train_lstm, y_test_lstm = prep.normalize_and_prepare_data(data['close'].values, scaler)

x_train.shape =  (3357, 20, 1)
y_train.shape =  (3357, 1)
x_test.shape =  (824, 20, 1)
y_test.shape =  (824, 1)


In [29]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [30]:
model = torch.load('lstm_mrf.pt')
model.eval()

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [31]:
y_train_pred = model(x_train)

In [32]:
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))

In [33]:
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# lstm.append(trainScore)
# lstm.append(testScore)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
train_gt = y_train[:,0]
train_preds = y_train_pred[:,0]

In [ ]:
train_preds_df = pd.DataFrame({
    'Train GT': train_gt,
    'Train preds': train_preds
})

train_preds_df.head()

,Train GT,Train preds
0,307.649994,310.843445
1,309.000000,308.395264
2,304.100006,307.827850
3,300.050018,306.336426
4,300.399994,303.671539


In [ ]:
train_preds_df['diff'] = (train_preds_df['Train GT'] - train_preds_df['Train preds']).abs() 

In [ ]:
train_preds_df.head()

,Train GT,Train preds,diff
0,307.649994,310.843445,3.193451
1,309.000000,308.395264,0.604736
2,304.100006,307.827850,3.727844
3,300.050018,306.336426,6.286407
4,300.399994,303.671539,3.271545


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_preds_df.index, y=train_preds_df['Train GT'], mode='lines', name='training ground truth'))
fig.add_trace(go.Scatter(x=train_preds_df.index, y=train_preds_df['Train preds'], mode='lines', name='training preds'))
fig.add_trace(go.Scatter(x=train_preds_df.index, y=train_preds_df['diff'], mode='lines', name='training error'))

In [ ]:
train_preds_df.to_csv('MRF_ErrorDistribution.csv', index=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_preds_df.index[1850:1900], y=train_preds_df['Train GT'][1850:1900], mode='lines', name='training ground truth'))
fig.add_trace(go.Scatter(x=train_preds_df.index[1850:1900], y=train_preds_df['Train preds'][1850:1900], mode='lines', name='training preds'))
fig.add_trace(go.Scatter(x=train_preds_df.index[1850:1900], y=train_preds_df['diff'][1850:1900], mode='lines', name='training error'))
